In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree

In [2]:
main_url = 'https://aruba-realty.com/'
sub_links = ["property/list/houses-for-sale", "property/list/longterm-rentals",
             "property/list/vacation-rentals", "property/list/condominiums-for-sale", "property/list/commercial-for-sale",
             "property/list/property-land-for-sale", "property/list/commercial-rentals", "property/list/project-development"]

In [3]:
def join_links(main, sub):
    links  = []
    for sub in sub_links:
        url = main_url + sub
        links.append(url)
    return links
links = join_links(main_url, sub_links)

In [4]:
links

['https://aruba-realty.com/property/list/houses-for-sale',
 'https://aruba-realty.com/property/list/longterm-rentals',
 'https://aruba-realty.com/property/list/vacation-rentals',
 'https://aruba-realty.com/property/list/condominiums-for-sale',
 'https://aruba-realty.com/property/list/commercial-for-sale',
 'https://aruba-realty.com/property/list/property-land-for-sale',
 'https://aruba-realty.com/property/list/commercial-rentals',
 'https://aruba-realty.com/property/list/project-development']

In [5]:
def get_all_url(li):
    all_links = []
    for link in li:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, "html.parser")
        anchors = soup.find('div', {'class': 'property-type-grid'})
        for anchor in anchors:
            links  = anchor.find_all('a', href = True)
            for link in links:
                urls = link.get('href')
                #print(urls)
                all_links.append(urls)
    return all_links
path = get_all_url(links)

In [6]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        dom = etree.HTML(str(soup))
        
        name = soup.find('span', class_ = "listing-heading").get('title')
        location = soup.find('div', class_ = "address").text.strip()
        try:
            property_status = soup.find('div', class_ = "page-property-details__status").text
            property_type = soup.find('div', class_ = "listing-type").text
        except AttributeError:
            property_status =None
            property_type = None
        price = soup.find('div', class_ = "price").text.strip().split(" ")[1]
        rooms = soup.find_all("div", class_="listing-amenety")
        try:
            pool = rooms[2].text
            bedroom = rooms[0].text.split(" ")[0]
            bathroom = rooms[1].text.split(" ")[0]
            lat = dom.xpath('///*[@id="map"]/script')[0].text.strip().split('{')[2].split(',')[0].split(':')[1].strip()
            long = dom.xpath('//*[@id="map"]/script')[0].text.strip().split('{')[2].split(',')[1].split(':')[1].strip().replace('}', "")
        except IndexError:
            pool = None
            bedroom = None
            bathroom=  None
            lat = None
            long = None
        link = link

        prop = {
            'name': name,
            'location': location,
            'property_status': property_status,
            'property_type':property_type,
            'price': price,
            'bedrooms': bedroom,
            'bathrooms' : bathroom,
            'Pool':pool,
            'Latitude': lat,
            'Longitude': long,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('data/aruba_reality.csv', header=True, index=False)

In [7]:
get_details(path)

Saving: CASA Distinct
Saving: Casa Coral Pyramid Penthouse
Saving: Casa Coral Shell 3-CD
Saving: Casa Coral Shell GF-C
Saving: CASA Atlantis Model H-I
Saving: CASA Atlantis Penthouses
Saving: CASA Blue Residence Penthouse PH 3-6
Saving: Casa Coral Shell 1-E/2-E LOFT
Saving: CASA Atlantis Model D
Saving: CASA Blue Residence Penthouse PH 1-2
Saving: CASA Blue Residence Penthouse TH-4
Saving: CASA Noble
Saving: Casa Corazón de La Villa Sol
Saving: CASA Atlantis Model C
Saving: CASA Atlantis Model B
Saving: CASA Oceania unit 124
Saving: CASA Atlantis Model A
Saving: Casa Coral Shell GF-B
Saving: CASA Fantastical
Saving: CASA Caribbean Deluxe
Saving: CASA To The Moon & Back
Saving: CASA Royalty
Saving: CASA Harbor View
Saving: CASA Tropical Escape
Saving: CASA Blue Residence unit 221
Saving: Casa Coral Shell 2-F
Saving: Casa Tera Grandi
Saving: CASA Blue Residence unit 121
Saving: CASA Majestic
Saving: CASA Just Right
Saving: CASA Aruban Sunset
Saving: CASA Tamarijn
Saving: Casa Palm
Saving